In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
train_data[['cabin-a', 'cabin-b', 'cabin-c']] = train_data['Cabin'].str.split('/',expand=True)
test_data[['cabin-a', 'cabin-b', 'cabin-c']] = test_data['Cabin'].str.split('/',expand=True)

In [4]:
data_columns = train_data.columns

X = train_data.drop("Transported",axis=1)
y = train_data["Transported"]
X_test = test_data

drop_feats=['PassengerId','Name','Cabin']
X = X.drop(drop_feats, axis=1)
X_test = X_test.drop(drop_feats,axis=1)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid= train_test_split(X,y,test_size=0.2)

### boolean vals need imputing -->  cryosleep, vip,
### break cabin into 3 cols    --> cabin-a, cabin-b,cabin-c
### categorical variables      --> homeplanet,destination, homeplanet


In [6]:
s = (X_train.dtypes=='object')
object_cols= list(s[s].index)
print(object_cols)

['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin-a', 'cabin-b', 'cabin-c']


In [7]:
cat_cols = [o for o in object_cols if X_train[o].dtype=='object' and (len(X_train[o].unique())>0 and len(X_train[o].unique())<5)]
num_cols = [o for o in X_train.columns if X_train[o].dtype=='float64' or X_train[o].dtype=='int64']
print("cats",cat_cols)
print("nums",num_cols)

cats ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin-c']
nums ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

In [9]:
numerical_transformer = SimpleImputer(strategy  = "constant")

categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocess =ColumnTransformer(
    transformers=[
    ('num',numerical_transformer,num_cols),
    ('cat',categorical_transformer,cat_cols)
])


In [10]:
# help(ColumnTransformer)
model= RandomForestRegressor(n_estimators=1000, random_state=0)

In [11]:
pipe_fin= Pipeline(steps=[
    ('preprocess',preprocess),
    ('model',model)
])

In [12]:
# pipe_fin.fit(X_train,y_train)

In [13]:
# pred = pipe_fin.predict(X_valid)

In [14]:
# maes=[]
# for i in range (0,10,1):
#     pred_1= 1*(pred>(i/10))
#     mae= mean_absolute_error(pred_1,y_valid)
#     maes.append(mae)
    

# thresh = maes.index(min(maes))

In [15]:
# thresh

#### CROSS-VALIDATION

score= -1 * cross_val_score(pipe_fin,X,y,
                          cv=4,
                          scoring='neg_mean_absolute_error')
print(score)

In [16]:
pipe_fin.fit(X,y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['HomePlanet', 'CryoSleep',
                                                   'Destination', 'VIP',
                     

In [17]:
res = pipe_fin.predict(X_test)
res = res>0.5
output= pd.DataFrame({'PassengerId':test_data['PassengerId'],
                     'Transported':res})
output.to_csv('submission.csv',index=False)
# output